## 경험적으로 추천해보기

### 수행단계
 - 데이터는 제공되는 파일인 prob-0101.csv를 사용한다.
 - 영화 개수, 배급사 수, 감독 수, 출연진 수, 장르 수를 파악한다.
 - 감독, 출연진 등 중복 조건들에 대해 파악한다.
 - 본인이 추천하고 싶은 기준을 설정한다.
 - 본인의 기준에 의해 영화 몇개를 추천하는 것이 적당한지 실험해 본다.
 - 다양한 접근 조회 방법을 실험한다. (감독, 출연진 중복, 장르별 등)
 - 개인별로 정한 기준이 서로 어떻게 다른지 확인한다.

In [1]:
# %pip install pandas
import pandas as pd
df = pd.read_csv('prob-0101.csv')
# print(df)
# df[['제목', '배급사']]

In [2]:
import re # 정규식 사용을 위한 모듈

print(f"영화 수: { len(df)}")

# 유니크한 갯수만 출력
print(f"중복 제외 배급사 수: { df['배급사'].nunique() }")
print(f"중복 제외 감독 수: { df['감독'].nunique() }")

# 쉼표(,) 로 구분된 출연진 파악 (ex. (1) 영화1, 영화2의 출연진 목록을 쉼표로 합치고 (2) 전체 합친 것을 쉼표로 나눔, (3) set으로 중복제거하고 (4) len으로 count)
print(f"중복 제외 출연진 수: { len(set(','.join(df['출연진'].dropna()).split(','))) }") # dropna(): 결측값(NaN)을 제거

# 정규식을 사용하여 가운데 점(·), 쉼표(,), 슬래쉬(/)를 전부 쉼표로 바꿈
print(f"중복 제외 장르 수: { len(set(re.split(r'[·,/]', ','.join(df['장르'].dropna())))) }")

영화 수: 59
중복 제외 배급사 수: 52
중복 제외 감독 수: 58
중복 제외 출연진 수: 268
중복 제외 장르 수: 17


In [3]:
# 추천 기준 점수
# 1. 자주 등장한 배우가 출연한 영화 (40%) 
# - 출연진 중복 점수를 각 배우에게 부여
# - 영화에 출연한 배우의 점수를 합산
# 2. 장르는 드라마와 액션 (40%)
# 3. 최근 개봉일수록 우선 (20%)


from collections import Counter

# 출연진 빈도 계산
all_actors = ','.join(df['출연진']).split(',')  # 중복 포함 전체 배우 목록
actor_frequency = Counter(all_actors) # 빈도 수 계산

# 영화별 출연진 중복 점수 계산
df['출연진_중복점수'] = df['출연진'].apply(lambda cast: sum(actor_frequency[actor] for actor in cast.split(',')))

# print(df[['출연진', '출연진_중복점수']])

# 개봉일 컬럼을 datetime 형식으로 변환
df['개봉일'] = pd.to_datetime(df['개봉일'], errors='coerce')

# 정규화 및 가중치 부여 함수
# 특정 Series 데이터를 0 ~ 1로 정규화하고 가중치를 곱해 반환
def normalize(series, weight):
    return (series - series.min()) / (series.max() - series.min()) * weight

In [4]:
# 추천 점수 계산
df['추천점수'] = (
    normalize(df['출연진_중복점수'], 0.4) +
    df['장르'].apply(lambda x: 1 if any(genre in ['드라마', '멜로'] for genre in re.split(r'[·,/]', x)) else 0) * 0.4 +
    normalize((df['개봉일'] - df['개봉일'].min()).dt.days, 0.2)
)

print(df[['제목', '추천점수']])

                          제목      추천점수
0                        스위치  0.080000
1                       강남좀비  0.421143
2                 별 볼일 없는 인생  0.409143
3                   10일간의 애인  0.409143
4                         교섭  0.456000
5                         유령  0.136000
6                         정이  0.058286
7             열여덟, 어른이 되는 나이  0.484000
8                      다음 소희  0.500000
9           어쩌면 우린 헤어졌는지 모른다  0.500000
10           우리 사랑이 향기로 남을 때  0.540000
11           스마트폰을 떨어뜨렸을 뿐인데  0.090286
12                       카운트  0.596000
13                        살수  0.136000
14                     컨버세이션  0.557143
15                       멍뭉이  0.484000
16                       대외비  0.104000
17                   그대 어이가리  0.512000
18                     소울메이트  0.520000
19                    낭만 캠퍼스  0.609143
20                   여섯 개의 밤  0.596000
21                  여덟 번째 감각  0.616000
22                       웅남이  0.276000
23                 찬란한 나의 복수  0.536000
24                       

In [5]:
# 출연진 점수 출력
print("\n출연진과 출연진 중복 점수:")
print(df[['출연진', '출연진_중복점수']])

# 장르 점수 출력
print("\n장르와 관련된 점수:")
df['장르_점수'] = df['장르'].apply(
    lambda x: 1 if any(genre in ['드라마', '액션'] for genre in re.split(r'[·,/]', x)) else 0 # 정규식 사용
) * 0.4
print(df[['장르', '장르_점수']])

# 개봉일 점수 출력
print("\n개봉일과 개봉일 관련 점수:")
df['개봉일_점수'] = normalize((df['개봉일'] - df['개봉일'].min()).dt.days, 0.2)
print(df[['개봉일', '개봉일_점수']])



출연진과 출연진 중복 점수:
                                                  출연진  출연진_중복점수
0                              권상우, 오정세, 이민정, 박소이, 김준         5
1                                             지일주, 지연         2
2                                                 정가은         1
3                                                 송민경         1
4                                        황정민, 현빈, 강기영         3
5                        설경구, 이하늬, 박소감, 박해수, 서현우, 김동희         7
6                                       강수연, 김현주, 류경수         3
7                                  임선우, 김명찬, 이장유, 박현숙         4
8                                  김시은, 배두나, 박우영, 정수하         4
9                                  이동휘, 정은채, 강길우, 정다은         4
10                       윤시윤, 설인아, 노상현, 문지인, 이규복, 김영웅         6
11                                      천우희, 임시완, 김희원         3
12                       진선규, 성유빈, 오나라, 고창석, 장동주, 고규필         8
13                            신현준, 이문식, 김민경, 홍은기, 최성원         5
14                     

In [6]:
# 추천 영화 정렬 및 상위 3개 출력
recommended_movies = df.sort_values(by='추천점수', ascending=False).head(3)
print(recommended_movies[['제목', '감독', '출연진', '장르', '추천점수']])

        제목   감독                                               출연진          장르  \
39      드림  이병헌  박서준, 아이유, 김종수, 고창석, 정승길, 이현우, 양현민, 홍완표, 허준석, 이하늬         드라마   
53     실버맨  최윤호                           김정팔, 박상욱, 신의현, 이경준, 유재인         드라마   
29  솔라 플라워  한경탁        오원빈, 서지희, 심현탁, 박정철, 김수하, 김민, 김봉주, 조수빈, 정병훈  멜로,로맨스,드라마   

        추천점수  
39  0.788000  
53  0.684000  
29  0.665143  
